# Loading (made by Dengsong Wang)

In [22]:
import psycopg2

# change dbname, user, password, host, port to your own if you want to run it on your own database
def create_conn():
    connection = psycopg2.connect(
        dbname="4142_Database",
        user="postgres",
        password="huazaiwang123456",
        host="localhost",
        port="5432"
    )
    return connection

conn = create_conn()

In [3]:
import pandas as pd

fact_table = pd.read_csv("../transform/fact_table.csv")
fact_table.dtypes

Surrogate Keys      int64
location           object
date               object
Tm Average        float64
Tx Average        float64
Tn Average        float64
P Average         float64
S_G Average       float64
total             float64
house             float64
land              float64
dtype: object

In [4]:
date_dimension = pd.read_csv("../transform/date_dimension.csv")
location_dimension = pd.read_csv("../transform/location_dimension.csv")
real_estate_dimension = pd.read_csv("../transform/real_estate.csv")
regional_weather_summaries = pd.read_csv("../transform/regional_weather_summaries.csv")
weather_station_summary = pd.read_csv("../transform/weather_station_summary.csv")


In [24]:
# create a table 
def create_table(connection):
    cursor = connection.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS "4142_table" (
                          "Surrogate Keys" INTEGER PRIMARY KEY,
                          location TEXT,
                          date TIMESTAMP,
                          "Tm Average" FLOAT,
                          "Tx Average" FLOAT,
                          "Tn Average" FLOAT,
                          "P Average" FLOAT,
                          "S_G Average" FLOAT,
                          total FLOAT,
                          house FLOAT,
                          land FLOAT);''')
    connection.commit()
    cursor.close()
    
create_table(conn)
conn.close()

In [26]:
# load the df into the table
from sqlalchemy import create_engine

def load_dataframe(dataframe, table_name, connection):
    engine = create_engine(f"postgresql+psycopg2://{connection.info.user}:{connection.info.password}@{connection.info.host}:{connection.info.port}/{connection.info.dbname}")
    dataframe.to_sql(table_name, engine, if_exists='replace', index=False, schema='public', method='multi')


conn = create_conn()
load_dataframe(fact_table, '4142_table', conn)
conn.close()

In [27]:
# validate the df is loaded successfully
def show_table(table,connection):
    query = f"SELECT * FROM \"{table}\""
    dataframe = pd.read_sql_query(query, connection)
    connection.close()
    return dataframe

conn = create_conn()
table_df = show_table('4142_table',conn)
conn.close()

C:\Users\Dengsong\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
